In [10]:
import requests
import pandas as pd
import numpy as np
import functools 
import math
import json

In [11]:
df = pd.read_csv('data/econ.csv', index_col=False)

In [12]:
df.head()

,B19025_001E,B19025_001M,GEO_ID,NAME,B19101_001E,B19101_001M,B19101_002E,B19101_002M,B19101_003E,B19101_003M,...,S1701_C03_057E,S1701_C03_057M,S1701_C03_058E,S1701_C03_058M,S1701_C03_059E,S1701_C03_059M,S1701_C03_060E,S1701_C03_060M,S1701_C03_061E,S1701_C03_061M
0,8.195074e+08,33260066.0,0500000US36095,"Schoharie County, New York",7928,268,277,64,293,106,...,16.7,5.0,-888888888,-888888888,4.6,2.0,41.8,7.0,36.4,5.4
1,2.661527e+10,281519926.0,0500000US36005,"Bronx County, New York",326296,2480,35225,1291,23720,1264,...,35.9,1.9,-888888888,-888888888,5.3,0.5,42.3,1.7,64.4,1.1
2,1.423243e+10,188307367.0,0500000US36067,"Onondaga County, New York",113541,1115,5098,462,3249,353,...,15.5,1.8,-888888888,-888888888,2.6,0.5,39.2,2.3,43.9,1.6
3,1.395835e+09,55793102.0,0500000US36035,"Fulton County, New York",14438,413,754,183,433,132,...,13.3,5.3,-888888888,-888888888,1.8,1.2,35.3,6.6,41.2,4.2
4,2.072981e+09,57860345.0,0500000US36019,"Clinton County, New York",20084,563,973,212,828,234,...,11.8,4.2,-888888888,-888888888,4.2,2.4,49.6,6.0,41.2,4.0


In [13]:
with open('data/econ_meta_lookup.json', 'r') as f:
    meta_lookup = json.load(f)

In [14]:
list(meta_lookup.keys())[:5]

['Pop25t29', 'F16pl', 'FPop0t5', 'CvNIPop2', 'MPop35t39']

In [43]:
meta_lookup['MdPop67t69']

['B01001_045', 'B01001_021']

In [44]:
[i for i in meta_lookup.keys() if 'MdPop' in i]

['MdPop67t69',
 'MdPop65t66',
 'MdPop21',
 'MdPop22t24',
 'MdPop15t17',
 'MdPop45t49',
 'MdPop35t39',
 'MdPop75t79',
 'MdPop10t14',
 'MdPop5t9',
 'MdPop60t61',
 'MdPop25t29',
 'MdPop18t19',
 'MdPop30t34',
 'MdPop_3',
 'MdPop85pl',
 'MdPop70t74',
 'MdPop40t44',
 'MdPop55t59',
 'MdPop50t54',
 'MdPop20',
 'MdPop62t64',
 'MdPop80t84',
 'MdPop0t4']

In [65]:
pop = {'MdPop0t4': [0, 5],
 'MdPop10t14': [10, 15],
 'MdPop15t17': [15, 18],
 'MdPop18t19': [18, 20],
 'MdPop20': [20, 21],
 'MdPop21': [21, 22],
 'MdPop22t24': [22, 25],
 'MdPop25t29': [25, 30],
 'MdPop30t34': [30, 35],
 'MdPop35t39': [35, 40],
 'MdPop40t44': [40, 45],
 'MdPop45t49': [45, 50],
 'MdPop50t54': [50, 55],
 'MdPop55t59': [55, 60],
 'MdPop5t9': [5, 10],
 'MdPop60t61': [60, 62],
 'MdPop62t64': [62, 65],
 'MdPop65t66': [65, 67],
 'MdPop67t69': [67, 70],
 'MdPop70t74': [70, 75],
 'MdPop75t79': [75, 80],
 'MdPop80t84': [80, 85],
 'MdPop85pl': [85, np.inf]}

In [66]:
sorted(pop)

['MdPop0t4',
 'MdPop10t14',
 'MdPop15t17',
 'MdPop18t19',
 'MdPop20',
 'MdPop21',
 'MdPop22t24',
 'MdPop25t29',
 'MdPop30t34',
 'MdPop35t39',
 'MdPop40t44',
 'MdPop45t49',
 'MdPop50t54',
 'MdPop55t59',
 'MdPop5t9',
 'MdPop60t61',
 'MdPop62t64',
 'MdPop65t66',
 'MdPop67t69',
 'MdPop70t74',
 'MdPop75t79',
 'MdPop80t84',
 'MdPop85pl']

In [71]:
def get_median(buckets, row):
    ordered = sorted(buckets.keys())
    N = row[ordered].sum()
    C = 0
    i = 0
    while C <= N/2:
        C += row[ordered[i]]
        i += 1
    L = buckets[ordered[i]][0]
    F = row[ordered[i]]
    W = buckets[ordered[i]][1] - buckets[ordered[i]][0]
    median = L + ((N/2) - C)*W
    print(L)

In [72]:
get_median(pop, df[df.GEO_ID=='0500000US36095'])

KeyError: "None of [Index(['MdPop0t4', 'MdPop10t14', 'MdPop15t17', 'MdPop18t19', 'MdPop20',\n       'MdPop21', 'MdPop22t24', 'MdPop25t29', 'MdPop30t34', 'MdPop35t39',\n       'MdPop40t44', 'MdPop45t49', 'MdPop50t54', 'MdPop55t59', 'MdPop5t9',\n       'MdPop60t61', 'MdPop62t64', 'MdPop65t66', 'MdPop67t69', 'MdPop70t74',\n       'MdPop75t79', 'MdPop80t84', 'MdPop85pl'],\n      dtype='object')] are in the [columns]"

### Formula:
https://www.census.gov/content/dam/Census/library/publications/2018/acs/acs_general_handbook_2018_ch08.pdf

In [17]:
# e --> estimate, m --> moe
def get_e(e):
    return sum(e)

def get_m(m):
    return math.sqrt(sum(map(lambda x: x**2, m)))

def get_c(e, m): 
    if e == 0:
        return ''
    else:
        return m/1.645/e*100

def get_p(e, agg_e):
    if agg_e == 0: 
        return ''
    else:
        return e/agg_e*100

def get_z(e, m, p, agg_e, agg_m):
    if p == 0:
        return ''
    elif p == 100:
        return ''
    elif agg_e == 0:
        return ''
    elif m**2 - (e*agg_m/agg_e)**2 <0:
        return math.sqrt(m**2 + (e*agg_m/agg_e)**2)/agg_e*100
    else: 
        return math.sqrt(m**2 - (e*agg_m/agg_e)**2)/agg_e*100

In [18]:
def find_total(variable, stat='E'):
    if variable[0] == 'B' or variable[0] == 'C' : 
        return f"{variable.split('_')[0]}_001{stat}"
    elif variable[0] == 'D': 
        return f"{variable.split('_')[0]}_0001{stat}"
    else: #S1810_C01_001M
        return f"{'_'.join(variable.split('_')[:2])}_001{stat}"

In [19]:
%%time
i = 'MdEFFTWrk'
variables = meta_lookup[i]
all_columns = list(df.columns)
e_variables = list(map(lambda x: all_columns.index(f'{x}E'), variables))
m_variables = list(map(lambda x: all_columns.index(f'{x}M'), variables))
total_e = find_total(variables[0], 'E')
total_m = find_total(variables[0], 'M')
dff = df.values

CPU times: user 79.8 ms, sys: 39.3 ms, total: 119 ms
Wall time: 121 ms


In [20]:
i

'MdEFFTWrk'

In [21]:
%%time
df.loc[:,f'{i}E'] = np.apply_along_axis(get_e, 1, dff[:, e_variables])
df.loc[:,f'{i}M'] = np.apply_along_axis(get_m, 1, dff[:, m_variables])

df.loc[:,f'{i}C'] = df.apply(lambda row: get_c(row[f'{i}E'], row[f'{i}M']), axis=1)

if len(variables) == 1 and f'{variables[0]}PE' in df.columns:
    '''
    If for some of the records PE is already calculated, 
    then take them directly and calculate PE for the rest
    '''
    df.loc[:,f'{i}P'] \
        = df.loc[df[f'{variables[0]}PE'].isna(), :]\
            .apply(lambda row: get_p(row[f'{i}E'], row[total_e]), axis=1)    
    
    df.loc[:,f'{i}P']\
        = df.loc[~df[f'{variables[0]}PE'].isna(), :]\
            .loc[:,f'{variables[0]}PE']
else: 
    df.loc[:,f'{i}P']\
        = df.apply(lambda row: get_p(row[f'{i}E'], row[total_e]), axis=1)

if len(variables) == 1 and f'{variables[0]}PM' in df.columns:
    '''
    If for some of the records PM is already calculated, 
    then take them directly and calculate PM for the rest
    '''
    df.loc[:,f'{i}Z']\
        = df.loc[df[f'{variables[0]}PM'].isna(), :]\
            .apply(lambda row: get_z(row[f'{i}E'], 
                                     row[f'{i}M'], 
                                     row[f'{i}P'], 
                                     row[total_e],
                                     row[total_m]), axis=1)

    df.loc[:,f'{i}Z']\
        = df.loc[~df[f'{variables[0]}PM'].isna(), :]\
            .loc[:,f'{variables[0]}PM']
else:
    df.loc[:,f'{i}Z']\
        = df.apply(lambda row: get_z(row[f'{i}E'], 
                                     row[f'{i}M'], 
                                     row[f'{i}P'], 
                                     row[total_e],
                                     row[total_m]), axis=1)

CPU times: user 443 ms, sys: 112 ms, total: 555 ms
Wall time: 574 ms


In [22]:
output_cols = sum([[i+'E', i+'M', i+'C', i+'P', i+'Z'] for i in ['MdEFFTWrk']], []) + [ 'DP03_0094PE', 'DP03_0094PM','GEO_ID', 'NAME']

In [23]:
def handle_median(df, meta_lookup):
    cols = meta_lookup.keys()
    if 'AvgHHSz' in cols: 
        df['AvgHHSz']
    if 'MdEFFTWrk' in cols:
        df['MdEFFTWrk']

In [24]:
df[output_cols]

,MdEFFTWrkE,MdEFFTWrkM,MdEFFTWrkC,MdEFFTWrkP,MdEFFTWrkZ,DP03_0094PE,DP03_0094PM,GEO_ID,NAME
0,38714,1824.0,2.864118,-888888888,-888888888,-888888888,-888888888,0500000US36095,"Schoharie County, New York"
1,37062,362.0,0.593764,-888888888,-888888888,-888888888,-888888888,0500000US36005,"Bronx County, New York"
2,43796,829.0,1.150679,-888888888,-888888888,-888888888,-888888888,0500000US36067,"Onondaga County, New York"
3,35622,1777.0,3.032517,-888888888,-888888888,-888888888,-888888888,0500000US36035,"Fulton County, New York"
4,40593,1672.0,2.503913,-888888888,-888888888,-888888888,-888888888,0500000US36019,"Clinton County, New York"
5,36743,1808.0,2.991286,-888888888,-888888888,-888888888,-888888888,0500000US36099,"Seneca County, New York"
6,47906,1545.0,1.960526,-888888888,-888888888,-888888888,-888888888,0500000US36083,"Rensselaer County, New York"
7,54713,2450.0,2.722135,-888888888,-888888888,-888888888,-888888888,0500000US36079,"Putnam County, New York"
8,39034,2554.0,3.977516,-888888888,-888888888,-888888888,-888888888,0500000US36033,"Franklin County, New York"
9,35697,1170.0,1.992454,-888888888,-888888888,-888888888,-888888888,0500000US36043,"Herkimer County, New York"


In [25]:
def handle_median(df, meta_lookup):
    if 'MdEFFTWrk' in meta_lookup.keys():
        df['MdEFFTWrk']

In [26]:
df['MdEFFTWrk']

KeyError: 'MdEFFTWrk'

In [62]:
df[output_cols].to_csv('data/econ_final.csv', index=False)